# Implementación de una red profunda


En esta implementación de Deep learning, nuestro objetivo es predecir la rotación de clientes para un determinado
banco, y de esta forma saber que clientes tienen mayor probabilidad de andonar el servicio bancario.
En conjunto de datos utilizados es relativamente pequeño y contiene 10000 filas con 14 columnas

Se está utilizando la distribución Anaconda y marcos como Theano, TensorFlow y Keras.
Keras está construido sobre Tensorflow y Theano, que funcionan como backends.



# Artificial Neural Network
 Se comienza isntalando las librerias y cargando los datos.

In [1]:
# Instalar Theano
!pip install theano

# Instalar Tensorflow
!pip install tensorflow

# Instalar Keras
!pip install keras

# Importar las librerias

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importar la base de datos

dataset = pd.read_csv('/content/REC017260.csv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668110 sha256=724554ead88707b5204b94ae6b8e54633247de157b72b0e79556fdbb323b8522
  Stored in directory: /root/.cache/pip/wheels/d9/e6/7d/2267d21a99e4ab8276f976f293b4ff23f50c9d809f4a216ebb
Successfully built theano


Se crea arrays con las características incluidas en el conjunto de datos y la variable objetivo que es la columna 14, etiquetada como "Exited"

In [2]:
# El aspecto inicial de los datos es como se muestra a continuación:

X = dataset.iloc[:,3:13].values #Db
print(X)

Y = dataset.iloc[:,13].values #valores de la variable "Exited"
print(Y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


Para realizar la codificación one-hot de variables categóricas se puede usar el código que usa la librería scikit-learn. Se importa la clase OneHotEncoder de sklearn.preprocessing y la librería numpy.
Se crea un array de ejemplo llamado categorias que contiene variables categóricas A,B,C,A,C. Este array se cnvierte en un matriz de una sola columna utilizando reshape(-1,1) para que tenda una forma adecuada para el procesamiento de OneHotEncoder.
Se crea una instancia de OneHotEncoder llamada encoder.
Se ajusta y transforma los datos categóricos utilizando el método fit_transform del objetivo encoder. Esto convierte las cetegorías en una representación one-hot, los datos codificados se convierten en una matriz densa (una matriz regular) utilizadno el método toarray().
Finalmente, se imprime la matriz densa que contiene los datos codificados.


In [3]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

# Supongamos que ya tienes el array X definido
# Ejemplo: X = np.array([...])

# Codificar las columnas específicas usando LabelEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

# Crear el transformador de columnas especificando las columnas categóricas
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), [1, 2])  # Índices de las columnas categóricas
    ],
    remainder='passthrough'  # Dejar las demás columnas tal cual
)

# Ajustar y transformar los datos
X = column_transformer.fit_transform(X)

# Eliminar la primera columna para evitar la trampa de variables ficticias
X = X[:, 1:]

print(X)


[[0.0 0.0 1.0 ... 1 1 101348.88]
 [0.0 1.0 1.0 ... 0 1 112542.58]
 [0.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [0.0 0.0 1.0 ... 0 1 42085.58]
 [1.0 0.0 0.0 ... 1 0 92888.52]
 [0.0 0.0 1.0 ... 1 0 38190.78]]


Se utiliza la función **train_test_split** de ScikitLear para dividir nuestros datos en conjuntos de entrenamiento y conjunto de prueba. Se mantiene el **ratio train_test_split** a la proporción **80:20**


In [4]:
# Dividir el dataset en Training set y Test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)


Se escalan los datos para que sean más representativos.

Se ajusta y transforma los datos de entrenamiento usando la función **StandardScaler**.

Se estandariza nuestra escala para que se utilice el mismo método ajustado para transformar / escalar datos de prueba.


In [5]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X


array([[0.0, 0.0, 1.0, ..., 1, 1, 101348.88],
       [0.0, 1.0, 1.0, ..., 0, 1, 112542.58],
       [0.0, 0.0, 1.0, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 1.0, ..., 0, 1, 42085.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 92888.52],
       [0.0, 0.0, 1.0, ..., 1, 0, 38190.78]], dtype=object)

Los datos ahora se escalan correctamente. Finalmente, ya tenemos listo el preprocesamiento de los datos.

Ahora, se comienza con nuestro modelo, se importan los módulos necesarios aquí. Se necesita el módulo **secuencial** para inicializar la red neuronal y el módulo **dense** para agregar las capas ocultas


In [6]:
# Importar la libreria Keras y sus paquetes

import keras
from keras.models import Sequential
from keras.layers import Dense

# Add a Flatten layer before your Dense layer
from keras.layers import Flatten


El nombre del modelo será **"Classifier"** ya que nuestro objetivo es clasificar la rotación de clientes.

Luego se usa el módulo secuencial para la inicialización


In [7]:
# Iniciamos la red neuronal

classifier = Sequential()
classifier.add(Flatten())

Se agrega las capas ocultas una por una usando la función **Dense**.

Nuesto primer parámetro es **output_dim**. Es la cantidad de nodos que agregamos a esta capa. **int** es la inicialización del gradiente estocástico descendiente.


En una red neuronal se asigna pesos a cada nodo. En la inicialización, los pesos deben estar cerca de cero y los inicialiamos aleatoriamente usando la función **uniform**.

El parámetro **input_dim** es necesario solo para la primera capa, ya que el modleo no concoce el úmero de nuestras variablesde enrada.

Aquí el número total de variables de entrada es 11. En la segunda capa, el modelo conoce automáticamente el número de variables de entrada de la primera capa oculta.


In [8]:
# Agregar la capa de entrada y la primera capa oculta:

input_dim = X_train.shape[1]  # Asegurarse de que input_dim sea el número correcto de características
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=input_dim))

# Se agrega la segunda capa oculta:

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Se agrega la capa de salida:

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Ahora se va a compilar nuestra red. Hasta ahora se ha agregado varias capas a nuestro clasificador.

Ahora se va a ir compilando utilizando el método de **compile**. Los argumentos agregados en el control de compilación final completan la red neuronal, por lo que se debe ser cuidadosos en este paso.


**Optimizer** Conjunto óptimo de pesos. **Denscenso de gradiente estocático (SGD)**. El SGD depende de la perdida por lo que nuesto segundo parámetro es la pérdida. Si nuestra variable dependiente es binaria, se usa la función de perdida logaritmica llamada **'binary_crossentropy'**, y si nuestra variable dependiente tiene más de dos categorias
en la salida, entonces usamos **'categorical_crossentropy'**.

Se quiere mejorar el rendimiento de nuestra red neuronal en función de la precisión por lo que agregamos métricas como precisión.


In [9]:
# Compilar red neuronal

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


**Se deben ajustar varios códigos en este paso**

Ajuste de la red neuronal al conjunto de entrenamiento

Ahora se entrena nuestro modelo en los datos de entrenamiento. Se usa el método **fit** para adaptarse a nuestro modelo.

También se optimiza los pesos para mejorar la eficiencia del modelo. Para esto, se tiene que actualizar los pesos.

El **Tamaño del lote (Batch size)** es el número de observaciones después de las cuales actualizamos los pesos.

La **Época (epoch)** es el número total de iteraciones. Los valores de tamaño de lote y época se eligen por el método de prueba y error.


In [10]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 50)

Epoch 1/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7919 - loss: 0.5634
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7987 - loss: 0.4249
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7909 - loss: 0.4247
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8087 - loss: 0.4192
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8237 - loss: 0.4278
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8342 - loss: 0.4070
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8284 - loss: 0.4157
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8289 - loss: 0.4146
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8387 - loss: 0.4070
Epoch 10/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8379 - loss: 0.3984
Epoch 11/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8303 - loss: 0.4113
Epoch 12/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

# Ya se está en condiciones de hacer predicciones y evaluar el modelo.

In [11]:
# Predecir los resultados Test set

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


El resultado de la predicción le dará la probabilidad de que el cliente abandole la empresa. Se convierte esa probabilidad en binario 0 y 1.

Se va a predecir sobre una observación:

Geography: Spain

Credit Score: 500

Gender: Female

Age: 40

Tenure: 3

Balance: 50000

Number of Products: 2

Has Credit Card: Yes

Is Active Member: Yes

In [13]:
# Todos estos valores se codifican, según vimos antes:

new_prediction = classifier.predict(sc.transform(np.array([[0,0.0,0,500,1,40,3,50000,2,1,1,40000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


array([[False]])

En el último paso evaluamos el rendimiento de nuestro modelo. Ya se tienen resultados originales y, por lo tanto, se puede construir una matriz de confusión para verificar la precisión del modelo.


In [14]:
# Se contruye la matriz de confusión:

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)
cm

array([[1537,   57],
       [ 285,  121]])

In [19]:
# A partir de la matriz de confusión, la precisión de nuestro modelo se puede clacular co
import numpy as np

# Extraer los valores
posicion1 = cm[0, 0]
posicion4 = cm[1, 1]

# Realizar el cálculo
resultado = ((posicion1 + posicion4) / 2000)*100

# Mostrar el resultado
print("El resultado es:", resultado)

# Se puede ver el resultado de la predicción con:

print(new_prediction)

El resultado es: 82.89999999999999
[[False]]


El resultado es que el cliente no nos abandona.